In [2]:
import huggingface_hub
huggingface_hub.notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Data Processing

In [4]:
import os
import cv2
import glob
import json
import torch
import numpy as np
import pandas as pd
from typing import *
from PIL import Image
import pybboxes as pbx
from shutil import copyfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation, rc

rc('animation', html='jshtml')
pd.set_option('display.max_columns', None)
Image.MAX_IMAGE_PIXELS = None
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.cuda.is_available()
print (device)

False


In [5]:
%%capture
!git lfs install
!git clone https://huggingface.co/datasets/datadrivenscience/ship-detection
''' #Using datasets library
from datasets import load_dataset
data = load_dataset("datadrivenscience/ship-detection") '''

In [ ]:
# Define data directories
ROOT_DIR = "/content/ship-detection"
train_dir = os.path.join(ROOT_DIR, 'train')
train_imgs = glob.glob(train_dir+'/*.png')

test_dir = os.path.join(ROOT_DIR, 'test')
test_imgs = glob.glob(test_dir+'/*.png')

In [ ]:
# convert image metadata to dataframe for easy manipulations

train_metadata = os.path.join(train_dir, 'metadata.jsonl') 
test_metadata = os.path.join(test_dir, 'metadata.jsonl')

with open(train_metadata, 'r+') as j_files:
  j_list = list(j_files)
  
bboxes = []
filename = []
categories = []

for j_fle in j_list:
  rslt = json.loads(j_fle)

  bboxes.append(rslt['objects']['bbox'])
  filename.append(rslt['file_name'])
  categories.append(rslt['objects']['categories'])


train_df = pd.DataFrame(
    {
        "filenames": filename,
        "bboxes": bboxes,
        "categories": categories,
    })

train_df = train_df.explode(['bboxes', 'categories']).reset_index(drop=True)
train_df.head()

,filenames,bboxes,categories
0,0.png,"[6690, 3599, 7164, 3850]",0
1,0.png,"[6772, 3386, 7039, 3546]",0
2,0.png,"[6705, 3291, 7060, 3485]",0
3,0.png,"[6230, 3442, 6597, 3647]",0
4,0.png,"[5501, 790, 5552, 868]",0


In [ ]:
# all the images doesn't have a unique size, 
## tghrtere will be a need for reshape
# NO Need to run this, since we're using YOLO

img_height = []
img_width = []
img_ar = [] # aspect_ratio

for i in train_imgs:
  img = cv2.imread(i)
  height, width, _ = img.shape
  ar = width / height

  img_height.append(width)
  img_width.append(height)
  img_ar.append(ar)

## merge with train df
img_df = pd.DataFrame({
    "filenames" : train_df['filenames'].unique().tolist(),
    "img_height" : img_height,
    "img_width" : img_width,
    "aspect_ratio": img_ar,
})

train_df = train_df.merge(img_df, how="left", on="filenames")
train_df.head()

,filenames,bboxes,categories,img_height,img_width,aspect_ratio
0,0.png,"[6690, 3599, 7164, 3850]",0,7574,5625,1.346489
1,0.png,"[6772, 3386, 7039, 3546]",0,7574,5625,1.346489
2,0.png,"[6705, 3291, 7060, 3485]",0,7574,5625,1.346489
3,0.png,"[6230, 3442, 6597, 3647]",0,7574,5625,1.346489
4,0.png,"[5501, 790, 5552, 868]",0,7574,5625,1.346489


In [ ]:
## test metadta to csv file

with open(test_metadata, 'r+') as j_files:
  j_list = list(j_files)
  
bboxes = []
filename = []
categories = []

for j_fle in j_list:
  rslt = json.loads(j_fle)

  bboxes.append(rslt['objects']['bbox'])
  filename.append(rslt['file_name'])
  categories.append(rslt['objects']['categories'])


test_df = pd.DataFrame(
    {
        "filenames": filename,
        "bboxes": bboxes,
        "categories": categories,
    })

test_df = test_df.explode(['bboxes', 'categories']).reset_index(drop=True)
test_df.head()

,filenames,bboxes,categories
0,0.png,"[100, 100, 110, 130]",0
1,0.png,"[80, 80, 90, 100]",0
2,1.png,"[100, 100, 110, 130]",0
3,1.png,"[80, 80, 90, 100]",0
4,10.png,"[100, 100, 110, 130]",0


In [ ]:
# all the images doesn't have a unique size, 
## tghrtere will be a need for reshape

img_height = []
img_width = []
img_ar = [] # aspect_ratio

for i in test_imgs:
  img = cv2.imread(i)
  height, width, _ = img.shape
  ar = width / height

  img_height.append(width)
  img_width.append(height)
  img_ar.append(ar)

## merge with train df
img_df = pd.DataFrame({
    "filenames" : test_df['filenames'].unique().tolist(),
    "img_height" : img_height,
    "img_width" : img_width,
    "aspect_ratio": img_ar,
})

test_df = test_df.merge(img_df, how="left", on="filenames")
test_df.head()

,filenames,bboxes,categories,img_height,img_width,aspect_ratio
0,0.png,"[100, 100, 110, 130]",0,1676,1144,1.465035
1,0.png,"[80, 80, 90, 100]",0,1676,1144,1.465035
2,1.png,"[100, 100, 110, 130]",0,4000,4000,1.000000
3,1.png,"[80, 80, 90, 100]",0,4000,4000,1.000000
4,10.png,"[100, 100, 110, 130]",0,1959,1263,1.551069


In [ ]:
def display_imgs_w_bbox(img_path, imgs_df):
  img = cv2.imread(img_path)
  file = img_path.split('/')[-1]
  fig, ax = plt.subplots(figsize=(25,25))
  ax.imshow(img);
  bboxes = imgs_df[imgs_df.filenames==file]['bboxes'].tolist()

  for b in bboxes:
    rect = patches.Rectangle((b[0], b[1]), b[2]-b[0], b[3]-b[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

  plt.show()

In [ ]:
display_imgs_w_bbox(train_imgs[-1], train_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
## viewing all images with boundary boxes

def draw_boxes(img_path, imgs_df):
  img = cv2.imread(img_path)
  file = img_path.split('/')[-1]
  bboxes = imgs_df[imgs_df.filenames==file]['bboxes'].tolist()
  H, W, _ = img.shape

  for b in bboxes:
    x1 = b[0]
    y1 = b[1]
    x2 = b[2]
    x3 = b[3]
    x, y, w, h = pbx.convert_bbox((x1,y1,x2,x3), from_type='voc',
                                  to_type='yolo', image_size=(W, H))
    x1 = int((x-w/2)*W)
    y1 = int((y-h/2)*H)
    x2 = int((x+w/2)*W)
    y2 = int((y+h/2)*H)

    cv2.rectangle(img, (x1, y1), (x2, y2), (255,0, 0),5)

  return img

In [ ]:
# animate
def create_animation(img_path, num):
  imgs = [draw_boxes(img_path[i], train_df) for i in range(num)]
  fig=plt.figure(figsize=(15, 20))
  im=plt.imshow(cv2.cvtColor(imgs[0],cv2.COLOR_BGR2RGB))
  plt.close()

  def animate_func(i):
    im.set_array(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2RGB))
    return [im]

  return animation.FuncAnimation(fig, animate_func, frames=len(imgs), interval=1000//2)

In [ ]:
# Create animation for 10 images
create_animation(train_imgs, 10)

In [ ]:
# converting to yolov8 format 
# id,xmin,ymin,xmax,ymax
# 0.png,6690.0,3599.0,7164.0,3850.0
# [6690, 3599, 7164, 3850]

In [ ]:
### YOLO data format. takes this:
## class x_centre y_centre width height.
# if no objects in image, no .txt file required for the image

In [ ]:
## split validation and test files
filenames = train_df.filenames.unique().tolist()

train_idx = np.random.choice(len(filenames), round(len(filenames)*0.8), replace =False)
val_idx = np.array(list(set(range(len(filenames))) - set(train_idx)))

splt_val_imgs = [filenames[i] for i in list(val_idx)]
splt_train_imgs = [filenames[i] for i in list(train_idx)]

In [ ]:
print (splt_val_imgs)

['129.png', '4.png', '134.png', '7.png', '10.png', '139.png', '12.png', '13.png', '140.png', '141.png', '142.png', '17.png', '18.png', '145.png', '146.png', '147.png', '23.png', '25.png', '157.png', '31.png', '163.png', '42.png', '43.png', '174.png', '49.png', '177.png', '51.png', '52.png', '180.png', '184.png', '57.png', '58.png', '187.png', '190.png', '193.png', '201.png', '203.png', '208.png', '84.png', '87.png', '218.png', '97.png', '233.png', '107.png', '241.png', '115.png', '243.png', '244.png', '245.png', '247.png', '121.png']


In [ ]:
## copy val imgs to another folder
val_dir = "/content/ship-detection/validation"
if not os.path.exists(val_dir):
  os.mkdir(val_dir)

for i in splt_val_imgs:
  copyfile(f"{train_dir}/{i}", f"{val_dir}/{i}")

In [ ]:
# remove val images from train set
for i in splt_val_imgs:
  os.remove(f"{train_dir}/{i}")

In [ ]:
len(os.listdir(val_dir))

In [ ]:
# convert to yolo format
def convert_to_yolo(file_names: List, imgs_dir: os.PathLike, imgs_df:pd.DataFrame):

  for file in file_names:
    file_ = file.split('.')[0]
    img_path = os.path.join(imgs_dir, file)
    save_dir = os.path.join(imgs_dir, f"{file_}.txt")

    H, W, _ = cv2.imread(img_path).shape
    with open(save_dir, "w+") as fle:
      bboxes = imgs_df[imgs_df.filenames==file]['bboxes'].tolist()
      for bbox in bboxes:
        x1 = bbox[0]
        y1 = bbox[1]
        x2 = bbox[2]
        y2 = bbox[3]

        x, y, w, h = pbx.convert_bbox((x1, y1, x2, y2), from_type='voc',
                                      to_type='yolo', image_size=(W,H))
        
        fle.write(f"{0} {x} {y} {w} {h}\n")

In [ ]:
## train conversion and val conversion

#convert_to_yolo(file_names: List, imgs_dir: os.PathLike, imgs_df:pd.DataFrame):
convert_to_yolo(splt_train_imgs, train_dir, train_df)
convert_to_yolo(splt_val_imgs, val_dir, train_df)


In [ ]:
## test converiosn to yolo format
test_filenames = test_df.filenames.unique().tolist()
convert_to_yolo(test_filenames, test_dir, test_df)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(draw_boxes("/content/ship-detection/train/197.png", train_df))
''' imgp = draw_boxes(train_imgs[0])
im=plt.imshow(cv2.cvtColor(imgp,cv2.COLOR_BGR2RGB)) '''

### Training

In [ ]:
## training

In [ ]:
import yaml
train_yml = {
    "path" : "/content/ship-detection",
    "train": "train",
    "val" : "validation",
    "test": "test",
    "nc": 1,
    "names": ['ship']
}

with open("train.yml", "w+") as y_fle:
  yaml.dump(train_yml, y_fle, default_flow_style=True)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## TensorBoard: Start with 'tensorboard --logdir runs/detect/yolov8-v1-20epc-ship-detection', view at http://localhost:6006/

In [ ]:
CFG = {
    'epochs': 5,
    'img_size' : 720,
    #'device': 'cpu',#'cuda',#
    'batch' : 8,
    'optimizer': 'Adam',
    'architecture' : 'yolov8',
}


' run = wandb.init(\n    project = "satalite-ship-detection",\n    tags = [\'object-detection\', \'competition\', \'yolov8\'],\n    save_code = True,\n    config = CFG,\n) '

In [ ]:
from ultralytics import YOLO
model = YOLO(model="yolov8n.pt")

trainer = model.train(
    data='train.yml',
    imgsz=CFG['img_size'],
    epochs=CFG['epochs'],
    #device=CFG['device'],# "cuda",
    batch = CFG["batch"],
    optimizer = CFG['optimizer'],
    name='yolov8-v1-5epc-ship-detection'
)

100%|██████████| 6.23M/6.23M [00:00<00:00, 66.2MB/s]
Ultralytics YOLOv8.0.74 🚀 Python-3.9.16 torch-2.0.0+cu118 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=train.yml, epochs=5, patience=50, batch=8, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8-v1-5epc-ship-detection, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize

FileNotFoundError: ignored

In [ ]:
!cp -r "/content/runs" "/content/gdrive/MyDrive/Models"

### Validation step

In [ ]:
vrelts = model.val()

In [ ]:
# Make prediciton with best model
from ultralytics import YOLO

best_model = YOLO("/content/gdrive/MyDrive/Models/runs/detect/yolov8-v1-5epc-ship-detection/weights/best.pt")
preds = best_model.predict(
    conf=0.1, source = "/content/ship-detection/test")


image 1/385 /content/ship-detection/test/0.png: 480x736 300 ships, 276.3ms
image 2/385 /content/ship-detection/test/1.png: 576x736 43 ships, 172.9ms
image 3/385 /content/ship-detection/test/10.png: 736x736 300 ships, 220.4ms
image 4/385 /content/ship-detection/test/100.png: 736x608 199 ships, 181.6ms
image 5/385 /content/ship-detection/test/101.png: 736x640 28 ships, 199.1ms
image 6/385 /content/ship-detection/test/102.png: 576x736 300 ships, 155.1ms
image 7/385 /content/ship-detection/test/103.png: 736x736 300 ships, 217.0ms
image 8/385 /content/ship-detection/test/104.png: 736x736 300 ships, 208.1ms
image 9/385 /content/ship-detection/test/105.png: 704x736 300 ships, 204.3ms
image 10/385 /content/ship-detection/test/106.png: 736x736 300 ships, 209.5ms
image 11/385 /content/ship-detection/test/107.png: 224x736 300 ships, 78.2ms
image 12/385 /content/ship-detection/test/108.png: 736x384 300 ships, 115.0ms
image 13/385 /content/ship-detection/test/109.png: 736x672 300 ships, 191.9ms
im

In [ ]:
sort_test_img = [f"/content/ship-detection/test/{i}.png" for i in range(386)]

test_preds = pd.DataFrame(columns=range(6))
for i in range(len(preds)):
    arri = pd.DataFrame(preds[i].boxes.boxes.cpu()).astype(float)
    path = sort_test_img[i]
    file = path.split('/')[-1]
    arri = arri.assign(file=file)
    arri = arri.assign(i=i)
    test_preds = pd.concat([test_preds,arri],axis=0)
test_preds.columns = ['xmin','ymin','xmax','ymax','confidence','class','id','i']
display(test_preds)

In [ ]:
test_preds = test_preds[["id","xmin","ymin","xmax","ymax","confidence"]].reset_index(drop=True)

In [ ]:
sample_sub = pd.DataFrame([i.split('/')[-1] for i in test_imgs],columns=["id"]).reset_index(drop=True)
sample_sub["label"] = "0 0 0 10 10"

In [ ]:
# Convert detections to string format
import tqdm
for i in tqdm.tqdm(range(len(sample_sub))):
    detections = []
    idd = sample_sub.loc[i,'id']
    df = test_preds[test_preds['id']==idd]
#     print(len(df))
    for i, row in df.iterrows():
        detection_str = f"{row['confidence']} {int(row['xmin'])} {int(row['ymin'])} {int(row['xmax'])} {int(row['ymax'])}"
        detections.append(detection_str)
    detections_str = ', '.join(detections)
    idx = list(sample_sub[sample_sub['id']==idd].index)
    if len(df)>0:
        sample_sub.loc[idx,"label"] = detections_str
#     break